# Importing required libraries

In [ ]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
import pandas as pd
from sklearn import datasets
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
import statsmodels.api as sm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# loading the dataset

In [ ]:
data = pd.read_csv("DataML.csv")
data

,Review,label
0,Excellent look,Recommended
1,Good product,Recommended
2,Nice dress,Recommended
3,LOVEd THIS clothS,Recommended
4,Really the product was as it is in the photo I...,Recommended
...,...,...
595,"Average,Poor quality, black color faded with 1...",Not Recommended
596,Fabric is not good not cottoandit's a mix of f...,Not Recommended
597,"Product Not justify to the price - overpriced,...",Not Recommended
598,"Be honest Amazon.,Not satisfied. Looks totall...",Not Recommended


# Making X and y variables

In [ ]:
X, y = data.Review, data.label

# stemming, lemmatisation, stop words removal

In [ ]:
documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

# bag of words model

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

# TFIDF Model

In [ ]:
# from sklearn.feature_extraction.text import TfidfTransformer
# tfidfconverter = TfidfTransformer()
# X = tfidfconverter.fit_transform(X).toarray()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Multi layer Perceptron Implementaion

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(max_iter=100)

In [ ]:
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (150,100,50),(200,250,100)],
    'activation': ['tanh', 'relu','logistic'],
    'solver': ['sgd', 'adam','lbfgs'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive','invscaling'],
}

In [ ]:
# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Best parameters found:
 {'solver': 'lbfgs', 'learning_rate': 'invscaling', 'hidden_layer_sizes': (150, 100, 50), 'alpha': 0.0001, 'activation': 'tanh'}
0.858 (+/-0.051) for {'solver': 'lbfgs', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (150, 100, 50), 'alpha': 0.05, 'activation': 'tanh'}
0.858 (+/-0.039) for {'solver': 'lbfgs', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (200, 250, 100), 'alpha': 0.0001, 'activation': 'relu'}
0.850 (+/-0.044) for {'solver': 'lbfgs', 'learning_rate': 'constant', 'hidden_layer_sizes': (50, 100, 50), 'alpha': 0.0001, 'activation': 'relu'}
0.852 (+/-0.031) for {'solver': 'adam', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (50, 50, 50), 'alpha': 0.0001, 'activation': 'relu'}
0.765 (+/-0.089) for {'solver': 'sgd', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (50, 100, 50), 'alpha': 0.05, 'activation': 'tanh'}
0.504 (+/-0.190) for {'solver': 'sgd', 'learning_rate': 'invscaling', 'hidden_layer_sizes': (50, 100, 50), 'alpha': 0.05, '

In [ ]:
y_pred = clf.predict(X_test)

from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
print('Results on the test set:')
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))

Results on the test set:
                 precision    recall  f1-score   support

Not Recommended       0.86      0.86      0.86        65
    Recommended       0.84      0.84      0.84        55

       accuracy                           0.85       120
      macro avg       0.85      0.85      0.85       120
   weighted avg       0.85      0.85      0.85       120

0.85
[[56  9]
 [ 9 46]]


In [ ]:
TN= 47
FP= 12
specificity= TN/(TN+FP)
print(specificity)

0.7966101694915254


In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
# X = vectorizer.fit_transform(documents).toarray()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [ ]:
clffff = MLPClassifier(hidden_layer_sizes=(150,100,50), max_iter=300,activation = 'relu',solver='adam',random_state=1).fit(X_train, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clffff, X, y, cv=10)
print('Cross-Validation Accuracy Scores', scores)
print(scores.mean())

Cross-Validation Accuracy Scores [0.73333333 0.83333333 0.81666667 0.81666667 0.85       0.91666667
 0.7        0.86666667 0.88333333 0.9       ]
0.8316666666666667


In [ ]:
y_pred = clffff.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[58  7]
 [ 9 46]]
                 precision    recall  f1-score   support

Not Recommended       0.87      0.89      0.88        65
    Recommended       0.87      0.84      0.85        55

       accuracy                           0.87       120
      macro avg       0.87      0.86      0.87       120
   weighted avg       0.87      0.87      0.87       120

0.8666666666666667


In [ ]:
TN= 46
FP= 7
specificity= TN/(TN+FP)
print(specificity)

0.8679245283018868


# XG Boost Implementation

In [ ]:
from xgboost import XGBClassifier

In [ ]:
import xgboost as xgb
import lightgbm as lgb
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold

In [ ]:
ITERATIONS=10

In [ ]:
bayes_cv_tuner = BayesSearchCV(
    estimator = xgb.XGBClassifier(
        n_jobs = 1,
        objective = 'binary:logistic',
        eval_metric = 'auc',
        silent=1,
        tree_method='approx'
    ),
    search_spaces = {
        "subsample":[0.5, 0.75, 1],
"colsample_bytree":[0.5, 0.75, 1],
"max_depth":[2, 6, 12],
"min_child_weight":[1,5,15],
"learning_rate":[0.3, 0.1, 0.03],
"n_estimators":[100],
 'reg_alpha': [1.1, 1.2, 1.3],
 "colsample_bylevel":[0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    },    
    scoring = 'roc_auc',
    cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=42
    ),
    n_jobs = 3,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = 42
)
def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    all_models.to_csv(clf_name+"_cv_results.csv")
    

In [ ]:
result = bayes_cv_tuner.fit(X_train, y_train, callback=status_print)

Model #1
Best ROC-AUC: 0.7859
Best params: OrderedDict([('colsample_bylevel', 0.6), ('colsample_bytree', 0.75), ('learning_rate', 0.3), ('max_depth', 6), ('min_child_weight', 5), ('n_estimators', 100), ('reg_alpha', 1.2), ('subsample', 0.75)])

Model #2
Best ROC-AUC: 0.7859
Best params: OrderedDict([('colsample_bylevel', 0.6), ('colsample_bytree', 0.75), ('learning_rate', 0.3), ('max_depth', 6), ('min_child_weight', 5), ('n_estimators', 100), ('reg_alpha', 1.2), ('subsample', 0.75)])

Model #3
Best ROC-AUC: 0.8766
Best params: OrderedDict([('colsample_bylevel', 0.7), ('colsample_bytree', 1.0), ('learning_rate', 0.03), ('max_depth', 6), ('min_child_weight', 1), ('n_estimators', 100), ('reg_alpha', 1.1), ('subsample', 1.0)])

Model #4
Best ROC-AUC: 0.8766
Best params: OrderedDict([('colsample_bylevel', 0.7), ('colsample_bytree', 1.0), ('learning_rate', 0.03), ('max_depth', 6), ('min_child_weight', 1), ('n_estimators', 100), ('reg_alpha', 1.1), ('subsample', 1.0)])

Model #5
Best ROC-AUC:

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
xgb_model = xgb.XGBClassifier()

In [ ]:
param_dist = {
        'n_estimators':range(80,200,4),
        'max_depth':range(2,15,1),
        'learning_rate':np.linspace(0.01,2,20),
        'subsample':np.linspace(0.7,0.9,20),
        'colsample_bytree':np.linspace(0.5,0.98,10),
        'min_child_weight':range(1,9,1)
}

In [ ]:
grid = GridSearchCV(xgb_model,param_dist,cv = 3,scoring = 'neg_log_loss',n_jobs = -1)


In [ ]:
best_estimator = grid.best_estimator_
print(best_estimator)
 #Output the accuracy of the optimal trainer
print(grid.best_score_)

AttributeError: ignored

In [ ]:
best_score = bayes_cv_tuner.best_score_
best_params = bayes_cv_tuner.best_params_
print("Best score: {}".format(best_score))
print("Best params: ")
for param_name in sorted(best_params.keys()):
    print('%s: %r' % (param_name, best_params[param_name]))

Best score: 0.8766311692111837
Best params: 
colsample_bylevel: 0.7
colsample_bytree: 1.0
learning_rate: 0.03
max_depth: 6
min_child_weight: 1
n_estimators: 100
reg_alpha: 1.1
subsample: 1.0


In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
# X = vectorizer.fit_transform(documents).toarray()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(xgboost , X, y, cv=10)
print('Cross-Validation Accuracy Scores', scores)
print(scores.mean())

Cross-Validation Accuracy Scores [0.66666667 0.73333333 0.83333333 0.83333333 0.81666667 0.88333333
 0.71666667 0.83333333 0.86666667 0.85      ]
0.8033333333333333


In [ ]:
y_pred = bayes_cv_tuner.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[45 20]
 [ 6 49]]
                 precision    recall  f1-score   support

Not Recommended       0.88      0.69      0.78        65
    Recommended       0.71      0.89      0.79        55

       accuracy                           0.78       120
      macro avg       0.80      0.79      0.78       120
   weighted avg       0.80      0.78      0.78       120

0.7833333333333333


In [ ]:
TN= 49
FP= 20
specificity= TN/(TN+FP)
print(specificity)

0.7101449275362319
